In [45]:
import pandas as pd
import re
import numpy as np

In [46]:
def merge():
    #This function takes data from across the different Illinois raw data sources and combines them into a single csv file, for each year.
    for yr in range(19,24):
        docstring = "~/Documents/Erdos/Proj/math_ed_project/data/IL" +str(yr) + "/IL" +str(yr) +"RawData.xlsx"
        outstring = "~/Documents/Erdos/Proj/math_ed_project/data/IL" +str(yr) + "/IL" +str(yr) +"_Combined_Ed_Demographic.csv"

        df1, df2, df3 = pd.read_excel(docstring, sheet_name=["General", "Financial", "SAT"]).values()

        df1dis = df1[df1["Type" ]== "District"]
        if yr != 23:
            df1 = df1[df1["School Type"].isin(["HIGH SCHOOL", "CHARTER SCH"])]
        else:
            df1 = df1[df1["School Type"].isin(["High School", "Middle/Junior High School"])]
        df1 = df1[df1["Type"] == "School"]

        # Step 1: Merge df1 with df1dis on "District Name"
        merged = df1.merge(
            df1dis,
            on="District",
            suffixes=("", "_df1dis"),
            how="left"
        )

        # Step 2: For each column in df1, fill in missing values using the value from df1dis. This is filling in missing school level data with corresponding district level data
        for col in df1.columns:
            if col != "District":  # Skip the join key
                col_df1dis = col + "_df1dis"
                if col_df1dis in merged.columns:
                    merged[col] = merged[col].fillna(merged[col_df1dis])

        # Step 3: Drop the extra _df1dis columns to clean up
        df1 = merged[df1.columns]
        del merged



        common_cols1 = set(df1.columns) & set(df2.columns) - {"RCDTS"}
        common_cols2 = set(df1.columns) & set(df3.columns) - {"RCDTS"}
        df2 = df2.drop(columns=common_cols1)
        df3 = df3.drop(columns=common_cols2)

        df1 = df1.merge(df2,on="RCDTS",how="inner")
        df1 = df1.merge(df3, on = "RCDTS", how = "inner")
        df1["Year"] = "20" + str(yr)
        

        df1.to_csv(outstring,index=False)





In [47]:


def instr(n):
        #Address each finle by year, with in-string
        return "~/Documents/Erdos/Proj/math_ed_project/data/IL" +str(n) +"/IL" +str(n) +"_Combined_Ed_Demographic.csv"

def cleaner():
    #This function standardizes the column names across all the data frames (they were a *mess* in the raw data)

    
    global df19
    df19 = pd.read_csv(instr(19))
    global df20
    df20 = pd.read_csv(instr(20))
    global df21
    df21 = pd.read_csv(instr(21))
    global df22
    df22 = pd.read_csv(instr(22))
    global df23
    df23 = pd.read_csv(instr(23))

    global dfs
    dfs = {
        'df19': df19,
        'df20': df20,
        'df21': df21,
        'df22': df22,
        'df23': df23
    }


   

    #Columns which were not available in every year. Remove, for consistency
    cols_to_remove = [
        "Chronically Truant Students",
        "Total Teacher Headcount",
        "% Students with an IEP - Nat Haw/Other Pac Isndr",
        "% Students with an IEP - Am Ind/Alaska Nat",
        "% Students with an IEP - Two or More Races","District Size",
        "Avg Class Size - All Grades"
        ]

    cols_to_fill_with_zero = [
        "% Student Enrollment - White",
        "% Student Enrollment - Black or African American",
        "% Student Enrollment - Hispanic or Latino",
        "% Student Enrollment - Asian",
        "% Student Enrollment - Native Hawaiian or Other Pacific Islander",
        "% Student Enrollment - American Indian or Alaska Native",
        "% Student Enrollment - Two or More Races",
        "% Student Enrollment - Children with Disabilities",
        "% Student Enrollment - EL",
        "% Student Enrollment - IEP",
        "% Student Enrollment - Low Income",
        "% Student Enrollment - Homeless",
        "% Students with an IEP - White",
        "% Students with an IEP - Black or African American",
        "% Students with an IEP - Hispanic or Latino",
        "% Students with an IEP - Asian"

    ]





    #Rename columns so they are consistent across years
    for name, df in dfs.items():
        for col in df.columns:
            if "Master" in col:
                df.rename(columns={col: "Percent Teachers With Masters Degree"}, inplace=True)
            if "Bachelor" in col:
                df.rename(columns={col: "Percent Teachers With Bachelor Degree"}, inplace=True)
            if "Attendace" in col:
                df.rename(columns={col: "Teacher Attendance Rate"}, inplace=True)
            if "SAT Math Average" in col:
                df.rename(columns={col: "SAT Math Average"}, inplace=True)
            if "SAT Reading Average" in col:
                df.rename(columns={col: "SAT Reading Average"}, inplace=True)
            if "Grades Served" in col:
                df.rename(columns= {col: "Grades Served"}, inplace=True)

            #Remove columns we don't want that appear in some dfs
            if col in cols_to_remove:
                df.drop(columns=col, inplace=True)

            #Some dfs leave 0s blank or filled with "*". Fill those with 0s.
            df[cols_to_fill_with_zero] = df[cols_to_fill_with_zero].fillna(0)
            df.replace('*', 0, inplace=True)
        





    #Standardize punctuation
    def clean_column_names(dfs, to_lower=False):
        for name, df in dfs.items():
            new_cols = []
            for col in df.columns:
                cleaned = col
                cleaned = cleaned.replace("–", "-").replace("—", "-")  # Replace smart dashes with hyphen
                cleaned = cleaned.replace("“", '"').replace("”", '"')  # Smart quotes to plain
                cleaned = cleaned.replace("‘", "'").replace("’", "'")  # Smart apostrophes
                cleaned = re.sub(r"\s+", " ", cleaned)                # Collapse multiple spaces
                cleaned = cleaned.strip()                             # Remove leading/trailing spaces
                if to_lower:
                    cleaned = cleaned.lower()
                new_cols.append(cleaned)
            df.columns = new_cols
            print(f"Cleaned column names for {name}")

    clean_column_names(dfs, to_lower=False)


In [48]:
# Still using the same dictionary of dataframes

def check():
    #Module to check which column names don't appear in every data frame year

    from collections import Counter, defaultdict

    # Step 1: Count all column names
    all_columns = [col for df in dfs.values() for col in df.columns]
    column_counts = Counter(all_columns)

    # Step 2: For each non-universal column, list which dataframes include it
    columns_with_sources = defaultdict(list)

    for col in column_counts:
        if column_counts[col] < 5:
            for name, df in dfs.items():
                if col in df.columns:
                    columns_with_sources[col].append(name)

    # Step 3: Print the results
    print("Columns that do NOT appear in all 5 dataframes:\n")
    for col, present_in in columns_with_sources.items():
        print(f"'{col}' appears in: {', '.join(present_in)}")

    




In [49]:
#Whip that data into shape!
merge()
cleaner()
check()

#Drop all rows without SAT data
for key in dfs:
    df = dfs[key]

    # Coerce non-numeric values to NaN
    df["SAT Math Average"] = pd.to_numeric(df["SAT Math Average"], errors='coerce')

    # Drop rows where the value is NaN or 0
    df = df.dropna(subset=["SAT Math Average"])
    df = df[df["SAT Math Average"] != 0]

    print(df.shape)
    # Reset index and store back in dict
    dfs[key] = df.reset_index(drop=True)


# First, ensure all three DataFrames have the right column as numeric
for df in [df21, df22, df23]:
    df["High School Dropout Rate - Total"] = pd.to_numeric(
        df["High School Dropout Rate - Total"], errors='coerce'
    )

# Create a mapping from RCDTS to dropout rate for df21 and df23
dropout_21 = df21.set_index("RCDTS")["High School Dropout Rate - Total"]
dropout_23 = df23.set_index("RCDTS")["High School Dropout Rate - Total"]

# Function to get average from df21 and df23 if both exist
def average_if_available(rcdts):
    val_21 = dropout_21.get(rcdts, np.nan)
    val_23 = dropout_23.get(rcdts, np.nan)
    if pd.notna(val_21) and pd.notna(val_23):
        return (val_21 + val_23) / 2
    else:
        return np.nan  # don't fill if we can't get both

# Apply this to missing rows in df22
mask_missing = df22["High School Dropout Rate - Total"].isna()
df22.loc[mask_missing, "High School Dropout Rate - Total"] = df22.loc[mask_missing, "RCDTS"].apply(average_if_available)

# Print how many we successfully filled
filled_count = mask_missing.sum() - df22["High School Dropout Rate - Total"].isna().sum()
print(f"Filled {filled_count} missing dropout rates in df22.")




Cleaned column names for df19
Cleaned column names for df20
Cleaned column names for df21
Cleaned column names for df22
Cleaned column names for df23
Columns that do NOT appear in all 5 dataframes:

(713, 51)
(691, 51)
(701, 51)
(700, 51)
(692, 52)
Filled 51 missing dropout rates in df22.


In [50]:
#Export nice data frames
df19 = dfs['df19']
df20 = dfs['df20']
df21 = dfs['df21']
df22 = dfs['df22']
df23 = dfs['df23']

df19.to_csv(instr(19),index=False)
df20.to_csv(instr(20),index=False)
df21.to_csv(instr(21),index=False)
df22.to_csv(instr(22),index=False)
df23.to_csv(instr(23),index=False)